<a href="https://colab.research.google.com/github/JakeANam/AIstudy24/blob/master/Ensemble_and_Tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
# 이거부터 설치하자! - 하고 나서 session 다시시작하고 다음 code부터 실행 -> 연결하고 하는 거 X
# pycharm에 없는 library? 설치하고 진행하자
!pip install xgboost
!pip install pycaret

Traceback (most recent call last):
  File "/usr/local/bin/pip3", line 5, in <module>
    from pip._internal.cli.main import main
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/main.py", line 11, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/autocompletion.py", line 10, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/main_parser.py", line 9, in <module>
    from pip._internal.build_env import get_runnable_pip
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/build_env.py", line 19, in <module>
    from pip._internal.cli.spinners import open_spinner
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/spinners.py", line 9, in <module>
    from pip._internal.utils.logging import get_indentation
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/util

In [1]:
# ensemble: 여러 단순한 모델을 결합하여 정확한 model를 만드는 방법
# 정형 data - 수치화된 값
# 비정형 data - 수치화가 불가능한 data - text, image, video, sound 등 -> 신경망 algorism
# random forest - 결정 tree를random하게 만들어서 숲을 만든 후 최종적인 예측
# 1000개의 sample이 들어있는 가방 안에서 100개를 뽑을 때 하나 뽑고 다시 넣기 - 중복된 sample을 뽑을 수 있다 -> bootstrap sample
# bootstrap - data set에서 중복을 허용해서 data sampling(ex. css에서 설정하는 id, class)

In [2]:
# 분류 모델인 RandomForestClassifier는 기본적으로 전체 특성 개수의 제곱근만큼 특성을 선택한다
# -> 즉, 4개의 특성이 있다면 node마다 2개를 random하게 선택해서 사용
# 다만, 회귀모델인 RandomForestRegressor는 전체 특성 사용

# sklearn의 random forest는 기본적으로 100개의 결정 tree를 이런 방식으로 훈련한다
# 그 다음 분류일 때는 각 tree의 class별 확률을 평균화해서 가장 높은 확률을 가진 class를 예측으로 삼는다
# 회귀일 때는 단순히 각 tree의 예측을 평균한다
# 분류: sample을 몇개의 class 중 하나로 분류하는 문제
# 회기: 임의의 어떤 숫자를 예측하는 문제

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')
wine.head()
# alcohol(도수), sugar(당도), pH(산도), 클래스(0이면 red, 1이면 white)
# data에서 whiter가 양성으로 1

data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

print(data)
print('-'*50)
print(target)

[[ 9.4   1.9   3.51]
 [ 9.8   2.6   3.2 ]
 [ 9.8   2.3   3.26]
 ...
 [ 9.4   1.2   2.99]
 [12.8   1.1   3.34]
 [11.8   0.8   3.26]]
--------------------------------------------------
[0. 0. 0. ... 1. 1. 1.]


In [4]:
# 훈련 set, 테스트 set 생성
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

print(train_input.shape, test_input.shape)

(5197, 3) (1300, 3)


In [5]:
from sklearn.model_selection import cross_validate # 교차 검증용
from sklearn.ensemble import RandomForestClassifier # 100개 결정 tree에 사용

rf = RandomForestClassifier(n_jobs=-1, random_state=42) # n_jobs=-1 -> 모든 cpu 사용
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
# return_train_score=True -> 검증 점수와 훈련 세트에 대한 점수도 return

print(np.mean(scores['train_score']))
print(np.mean(scores['test_score']))

0.9973541965122431
0.8905151032797809


In [6]:
# RandomForest : 결정 tree의 ensemble -> 결정트리가 제공하는 중요한 매개변수를 모두 제공한다
# 결정 tree의 큰 장점 중 하나인 특성 중요도를 계산한다
# RandomForest의 특성 중요도 = 각 결정 tree의 특성 중요도를 취합한 것이다
rf.fit(train_input, train_target)

RandomForestClassifier(n_jobs=-1, random_state=42)

In [7]:
print(rf.feature_importances_)

# 왼쪽 순서대로 알콜도수, 당도, pH
# 이 결과는 LogisticRefression 결과와 비교했을때 - 당도 값 감소, 알콜도수, pH 증가
# 당도의 중요도 감소, 알콜도수, pH의 중요도 증가
# RandomForest가 특성 일부를 랜덤하게 선택하여 결정 tree를 훈련, 즉 중복을 허용하기 때문에
# 하나의 특성에 과도하게 집중하지 않고 좀 더 많은 특서잉 훈련에 기여할 기회가 됨 -> 과대적합 감소

[0.23167441 0.50039841 0.26792718]


In [8]:
# RandomForestClassifier 기능 중 자체적으로 모델을 평가하는 점수를 얻을 수 있다
# OOB(out of bag)부트스트랩에 포함되지 않고 남은 sample -> 결정 tree 평가용(검증 set로 활용)
# oob_score = True -> RandomForest는 각 결정 tree의 oob 점수를 평균화하여 출력(oob_score_)
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)


RandomForestClassifier(n_jobs=-1, oob_score=True, random_state=42)

In [9]:
print(rf.oob_score_)
# 교차검증 점수와 비슷한 결과
# oob 점수를 사용하면 교차 검증을 대신할 수 있어 결과적으로 훈련 set에 더 많은 sample을 사용할 수 있다

0.8934000384837406


In [10]:
# extra tree: 100개의 결정 tree 훈련 -> bootstrap sample 사용 X - 전체 훈련 set 사용
# 대신 node를 분할할 때 가장 좋은 분할을 찾는 것이 아니라 무작위로 분할
# extra tree의 결정 tree splitter='random'
# 하나의 결정 tree에서 특성을 무작위로 분할한다면 성능이 낮아지겠지만 많은 tree를 ensemble하기 때문에 과대적합을 막고 검증 set의 점수를 높이는 효과가 있다

from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))
# RandomForestClassifier와 결과가 비슷하지만 속도가 더 빠르다

0.9974503966084433 0.8887848893166506


In [11]:
et.fit(train_input, train_target)

ExtraTreesClassifier(n_jobs=-1, random_state=42)

In [12]:
print(et.feature_importances_)
# 특성 중요도를 보면 결정 tree보다 당도에 대한 의존성이 작다

[0.20183568 0.52242907 0.27573525]


In [13]:
# Gradient Boosting (기울기)
# 깊이가 얕은 결정 tree를 사용해서 이전 tree의 오차를 보완하는 방식으로 ensenble
# sklearn의 GradientBoostingClassifier는 기본적으로 깊이가 3인 결정 tree 100개 사용 - 때문에 과대적합에 강하고 일반적인 높은 일반화 성능을 기대한다

In [14]:
# Gradient는 경사 하강법을 사용하여 tree를 ensemble에 추가
# 분류 - 로지스틱 손실 함수, 회귀에서는 평균 제급 오차함수 사용
# 경사 하강법 손실 함수를 산으로 정의하고 가장 낮은 곳으로 찾아 내려오는 과정

# 가장 낮은 곳으로 내려오는 방법은 모델의 가중치와 절편을 조금씩 바꾸는 것
# 결정 tree를 계속 추가하면서 가장 낮은 곳을 찾아 이동, 손실 함수의 낮은 곳으로 천천히 이동
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))
# 과대 적합 해결

0.8881086892152563 0.8720430147331015


In [15]:
gb = GradientBoostingClassifier(random_state=42, learning_rate=0.2, n_estimators=500)
# n_estimators = 500 -> 결정 tree개수를 500개로 늘림 (기본값 100)
# learning_rate-0.2 -> 학습률(기본값 0.1)

scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))
#검증 점수 차이가 10% 이상 나지 않으면 과대적합 아님!(5 ~ 7 정도가 이상적)

0.9464595437171814 0.8780082549788999


In [16]:
gb.fit(train_input, train_target)

GradientBoostingClassifier(learning_rate=0.2, n_estimators=500, random_state=42)

In [17]:
print(gb.feature_importances_)
# Gradiebt Boosting이 Random Forest 보다 조금 더 높은 성능이 있다만, 순서대로 tree를 추가하기 때문에 훈련 속도가 좀 느리다
# 구형 model인 GradientBoostingClassifier에는 n_jobs=-1 매개값이 없다

[0.15887763 0.6799705  0.16115187]


In [18]:
# Histogram 기반 Gradient Boosting (Histogram Gradient) - Gradient Boosting의 개선 version
# 입력 특성을 256구간으로 나눴다 -> node를 분할할 때 최적의 분할을 매우 빠르게 찾을 수 있다
# 특히 256구간 중에서 하나를 떼어놓고 누락된 값을 위해서 사용
# 2^8 - 1bite와 관련

# HistGradientBoostingClassifier는 기본 매개변수에서 안정적인 성능을 얻을 수 있다
# HistGradientBoostingClassifier에는 tree의 갯수를 지정하는데 n_estimators 대신 max_iter를 사용하여 성능을 향상한다
from sklearn.ensemble import HistGradientBoostingClassifier # 가장 최근에 나옴, 아직 개발 중, 분류용

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [19]:
from sklearn.inspection import permutation_importance
# 특성의 중요도 확인 - 특성을 하나씩 random하게 섞어서 model의 성능이 변화되는지 관찰

# histogram 기반의 훈련을 진행 -> 훈련 세트에서 특성 중요도를 계산해보자!
hgb.fit(train_input, train_target)

HistGradientBoostingClassifier(random_state=42)

In [20]:
result = permutation_importance(hgb, train_input, train_target, n_repeats=10, n_jobs=-1, random_state=42)
# n_repeats=10 -> Random하게 10번 섞기 (기본값은 5)

print(result.importances_mean)
# test set에서 특성 중요도
# 중요도, 평균, 표준편차

[0.08876275 0.23438522 0.08027708]


In [21]:
result = permutation_importance(hgb, test_input, test_target, n_repeats=10, n_jobs=-1, random_state=42)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [22]:
# [0.05969231 0.20238462 0.049     ] histogram test set에서 특성 중요도
# [0.08876275 0.23438522 0.08027708] histogram 훈련 결과
# [0.20183568 0.52242907 0.27573525] Extra Tree 이전 결과
# [0.23167441 0.50039841 0.26792718] RandomForest 이전 결과

In [27]:
# HistGradientBoostingRegressor - Histogram 기반 Gradient boosting의 회귀 version
# sklearn 말고도 GradientBoosting algorism을 구현한 library가 다수 존재
# XGBoost가 대표적, 가장 널리 쓰인다 - Cross_validation이 가능하다

from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9558403027491312 0.8782000074035686


In [29]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884


In [ ]:
# # machine learning 하고 해야할 일
# 1. data set 준비
# 2. 분류, 회귀
# 3. 사용할 method
# 4. velidate, 점수 확인
# 5. test_sample로 test